# Radial Average
## Importing libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import sixtrackwrap as sx

from parameters import *

## Computing

In [2]:
error_2 = {}
DA_2 = {}
count_matrix_2 = {}
average_matrix_2 = {}

alpha_preliminary_values = np.linspace(-1.0, 1.0, samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]

# Extracting the radiuses with theta1 = theta2 = 0.0

engine = sx.radial_scanner(
    alpha_values, 
    np.zeros(alpha_values.shape),
    np.zeros(alpha_values.shape),
    d_r,
    starting_step=starting_step
)
engine.scan(turn_sampling[0], turn_sampling[-1], batch_size=batch_size)
all_radiuses = engine.extract_DA(turn_sampling)


Active radiuses: 1025 / 1025
Sample size per active radius: 2
Expected execution time for step: nan
Time per single iteration: 2.1910466101111434e-05
r: 11.0 . Turns to do: 1000 . Min found: 1000
Active radiuses: 1025 / 1025
Sample size per active radius: 2
Expected execution time for step: 44.92
Time per single iteration: 2.1893588856952947e-05
r: 13.0 . Turns to do: 1000 . Min found: 1000
Active radiuses: 1025 / 1025
Sample size per active radius: 2
Expected execution time for step: 44.88
Time per single iteration: 2.1900506601101015e-05
r: 15.0 . Turns to do: 1000 . Min found: 1000
Active radiuses: 1025 / 1025
Sample size per active radius: 2
Expected execution time for step: 44.90
Time per single iteration: 2.189296536329316e-05
r: 17.0 . Turns to do: 1000 . Min found: 148
Active radiuses: 1025 / 1025
Sample size per active radius: 2
Expected execution time for step: 44.88
Time per single iteration: 2.1914352556554284e-05
r: 19.0 . Turns to do: 1000 . Min found: 79
Active radiuses:

In [3]:
values = []
for i in tqdm(range(len(turn_sampling))):
    radiuses = all_radiuses[:, i]

    r, a, th1, th2 = sx.full_track_particles(
        radiuses,
        alpha_values,
        np.zeros(alpha_values.shape),
        np.zeros(alpha_values.shape),
        turn_sampling[i])

    count_matrix, avg_matrix, temp_values = sx.accumulate_and_return(r, a, th1, th2, n_subdivisions)
    
    values.append(temp_values)
    count_matrix_2[(samples, turn_sampling[i])] = count_matrix
    average_matrix_2[(samples, turn_sampling[i])] = avg_matrix

steps = [1]
while True:
    if (values[0].shape[0] - 1) / steps[-1] > 4:
        steps.append(steps[-1] * 2)
    else:
        break

for jump in steps:
    DA = []
    error = []
    for i in range(len(turn_sampling)):
        DA.append(integrate.romb(values[i][::jump], d_preliminar_alpha * jump) * 0.5)
        temp = integrate.romb(values[i][::jump * 2], d_preliminar_alpha * jump * 2) * 0.5
        error.append(np.absolute(DA[-1] - temp))
    DA_2[len(values[i][::jump])] = DA
    error_2[len(values[i][::jump])] = error

100%|██████████| 100/100 [41:45<00:00, 25.06s/it] 


## Saving data

In [4]:
with open("data/DA_2.pkl", 'wb') as f:
    pickle.dump(DA_2, f, protocol=4)
    
with open("data/error_2.pkl", 'wb') as f:
    pickle.dump(error_2, f, protocol=4)
    
with open("data/count_matrix_2.pkl", 'wb') as f:
    pickle.dump(count_matrix_2, f, protocol=4)
    
with open("data/avg_matrix_2.pkl", 'wb') as f:
    pickle.dump(average_matrix_2, f, protocol=4)